In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('second-query').getOrCreate()


In [2]:
user_data_columns = ['user-id','item-id','rating','timestamp']
user_data_df = spark.read.csv('u.data',sep="\t")

from functools import reduce
oldColumns = user_data_df.columns

user_data_df = reduce(lambda df,ids: df.withColumnRenamed(oldColumns[ids],user_data_columns[ids]),range(len(oldColumns)),user_data_df)

In [3]:
items_df = spark.read.csv('u.item',sep='|')

item_old_columns = items_df.columns
item_new_columns = ['item-id','movie-title','release-date','video-release-data','IMDB-url','unknown','action','adventure','animation',"children's",'comedy','crime','documentary','drama','fantasy','film-noir','horror','musical','mystery','romance','sci-fi','thriller','war','western']


items_df = reduce(lambda df,ids: df.withColumnRenamed(item_old_columns[ids],item_new_columns[ids]),range(len(item_old_columns)),items_df)

In [4]:
rating_df = items_df.join(user_data_df,on='item-id')

In [5]:
rating_df.show()

+-------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+------+---------+
|item-id|         movie-title|release-date|video-release-data|            IMDB-url|unknown|action|adventure|animation|children's|comedy|crime|documentary|drama|fantasy|film-noir|horror|musical|mystery|romance|sci-fi|thriller|war|western|user-id|rating|timestamp|
+-------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+------+---------+
|    242|        Kolya (1996)| 24-Jan-1997|              null|http://us.imdb.co...|      0|     0|        0|        0|         0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|  

In [6]:
rating_df = rating_df.groupBy('item-id','movie-title').agg({'rating':'count'})

In [7]:
top_20_names=rating_df.orderBy(rating_df['count(rating)'].desc()).limit(20)

In [8]:
top_20_names.show()

+-------+--------------------+-------------+
|item-id|         movie-title|count(rating)|
+-------+--------------------+-------------+
|     50|    Star Wars (1977)|          583|
|    258|      Contact (1997)|          509|
|    100|        Fargo (1996)|          508|
|    181|Return of the Jed...|          507|
|    294|    Liar Liar (1997)|          485|
|    286|English Patient, ...|          481|
|    288|       Scream (1996)|          478|
|      1|    Toy Story (1995)|          452|
|    300|Air Force One (1997)|          431|
|    121|Independence Day ...|          429|
|    174|Raiders of the Lo...|          420|
|    127|Godfather, The (1...|          413|
|     56| Pulp Fiction (1994)|          394|
|      7|Twelve Monkeys (1...|          392|
|     98|Silence of the La...|          390|
|    237|Jerry Maguire (1996)|          384|
|    117|    Rock, The (1996)|          378|
|    172|Empire Strikes Ba...|          367|
|    222|Star Trek: First ...|          365|
|    204|B